In [1]:
# Imports
import numpy as np

In [2]:
# Import the tools
from sys import path
path.append('../tools')
from tools import gen_inputs_outputs, logsig, lin, deriv_logsig, deriv_lin, e2, hardlim, deriv_hardlim

In [3]:
# Define the size of the network
# Order by layers (input, layer1, layer2, ...)
# !!! No output layer yet, wait to gen inputs
layer_sizes = [4, 7]

In [4]:
# Gen inputs, outputs and size of the last layer
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

In [5]:
# Append the output layer size
layer_sizes = np.array(layer_sizes + [S], dtype=int)

In [6]:
# Network shape
layer_sizes

array([4, 7, 3])

In [7]:
# Inputs
inputs

array([[0, 0, 0, 0],
       [0, 0, 0, 1],
       [0, 0, 1, 0],
       [0, 0, 1, 1],
       [0, 1, 0, 0],
       [0, 1, 0, 1],
       [0, 1, 1, 0],
       [0, 1, 1, 1],
       [1, 0, 0, 0],
       [1, 0, 0, 1],
       [1, 0, 1, 0],
       [1, 0, 1, 1],
       [1, 1, 0, 0],
       [1, 1, 0, 1],
       [1, 1, 1, 0],
       [1, 1, 1, 1]])

In [8]:
# Outputs
outputs

array([[0, 0, 0],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [0, 0, 1],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [0, 1, 0],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [0, 1, 1],
       [1, 0, 0],
       [1, 0, 1],
       [1, 1, 0]])

In [9]:
# Intialize the weights and biases randomly
# Also create the list to store the outputs of each layer
# Store the outptut for each layer
# Finally create the list of the sensitivities
weights_list = []
biases_list = []
n_list = []
a_list = []
s_list = []

In [10]:
# Define a function to reset all the parameters
# This is done to test multiple different network types
def reset_params():
    weights_list.clear()
    biases_list.clear()
    n_list.clear()
    a_list.clear()
    s_list.clear()
    for i in range(1, len(layer_sizes)):
        weights_list.append(np.random.rand(layer_sizes[i], layer_sizes[i-1]))
        biases_list.append(np.random.rand(layer_sizes[i], 1))
        n_list.append(np.empty((layer_sizes[i], 1), dtype=float))
        a_list.append(np.empty((layer_sizes[i], 1), dtype=float))
        s_list.append(np.empty((layer_sizes[i], 1), dtype=float))
    a_list.append(np.empty((layer_sizes[0], 1), dtype=float))

In [11]:
# Define the transfer functions
transfer_functions = [logsig, lin]

In [12]:
# Define the vectors with the derivatives of the transfer functions
# These need to be converted to dialation matrices
# but this is done after the partial derivatives are calculated
# so that the numerical values can be multiplied by the identity matrix
# makes my life easier
deriv_transfer_functions = [deriv_logsig, deriv_lin]

In [13]:
# Define a function to calculate the output of the network
# while also saving the data
# Assuming that the input is already a Rx1 matrix
def run_network(X):
    # Get the input for the first layer so that
    # no need to use X and can calculate
    # recursively
    n_list[0] = np.matmul(weights_list[0], X) + biases_list[0]
    a_list[0] = transfer_functions[0](n_list[0])
    
    # This is important for the training later
    a_list[-1] = X
    
    # Calculate the rest of the outptut
    for i in range(1, len(weights_list)):
        n_list[i] = np.matmul(weights_list[i], a_list[i-1]) + biases_list[i]
        a_list[i] = transfer_functions[i](n_list[i])

In [14]:
# Calculate the senstivites for the last layer
# Write a function to fill the diagonal of the matrix
def fill_F_dot(layer_num):
    a = np.identity(len(n_list[layer_num]), float)
    np.fill_diagonal(a, deriv_transfer_functions[layer_num](n_list[layer_num]).flatten())
    return a

def train_network(learning_rate=.1, epochs=1):
    for i in range(epochs):
        # Iterate through the inputs and outputs
        for i in range(len(inputs)):
            # Run the network
            run_network(inputs[i].reshape(layer_sizes[0], 1))

            # Get the dialation matrix for the last layer
            F_dot = fill_F_dot(-1)

            # Calculate the sensitivites for the last layer
            s_list[-1] = -2 * np.matmul(F_dot, outputs[i].reshape(layer_sizes[-1], 1)-a_list[-2])

            # Update the weights and biases for the last layer
            weights_list[-1] += -learning_rate*np.matmul(s_list[-1], a_list[-3].T)
            biases_list[-1] += -learning_rate*s_list[-1]

            # Iterate through the remaining layers backwards
            for i in range(len(n_list)-2, -1, -1):
                s_list[i] = np.matmul(np.matmul(fill_F_dot(i), weights_list[i+1].T), s_list[i+1])
                weights_list[i] += -learning_rate*np.matmul(s_list[i], a_list[i-1].T)
                biases_list[i] += -learning_rate*s_list[i]

In [46]:
# Function to evaluate the performance
# of the network
def performance():
    results = np.empty((len(inputs), layer_sizes[-1]), dtype=float)
    for i in range(len(inputs)):
        run_network(inputs[i].reshape(layer_sizes[0], 1))
        results[i] = outputs[i] - a_list[-2].flatten()
    return e2(results)

In [16]:
# Print performance
def print_performance_dict(performance_dict):
    for a in sorted(performance_dict.items(), key=lambda x: x[1]):
        print(f'Error: {a[1]} | ' + a[0])

In [17]:
# Trains netowrk using the parameters for each bit size
def update_performance_dict(layer_sizes_list, performance_dict):
    for layer_sizes in layer_sizes_list:
        # Get the inputs and outputs for the specifiq network size
        inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])
        layer_sizes.append(S)
    
        # Iterate through the different learning rates
        # and epochs
        for rate in learning_rates:
            for epochs in epochs_list:
                # Reset weights and biases
                reset_params()
                # Train the network
                train_network(learning_rate=rate, epochs=epochs)
                # Update the performance dict
                performance_dict.update({f'Layers: {layer_sizes}, Learning rate: {rate}, Epochs: {epochs}': performance()})

In [18]:
### !!! Do not run code underneath until saving to the file
### It trains 22000 different networks
# Performance dictionnaries of the network
performance_dict_2bits = {}
performance_dict_4bits = {}
performance_dict_8bits = {}

In [19]:
# Parameters that won't change despite differnt layer sizes
# Set different learning rates
learning_rates = [1/(16*(2**i)) for i in range(10)]

# Set the diffrent epochs
epochs_list = [i for i in range(1, 21)]

In [20]:
# Multiple different layer sizes
# Start with 1 hidden layer
layer_sizes_list_2bits = [[4, 2*(2**i)]for i in range(10)]
layer_sizes_list_4bits = [[8, 2*(2**i)]for i in range(10)]
layer_sizes_list_8bits = [[16, 2*(2**i)]for i in range(10)]

In [21]:
# Calculate performance for 1 hidden layer
# 2 bits
update_performance_dict(layer_sizes_list_2bits, performance_dict_2bits)
# 4 bits
update_performance_dict(layer_sizes_list_4bits, performance_dict_4bits)
# 8 bits
update_performance_dict(layer_sizes_list_8bits, performance_dict_8bits)

In [22]:
# 2 Hidden layers
# Redefine the transfer functions
transfer_functions = [logsig, logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_logsig, deriv_lin]

# New layer sizes
layer_sizes_list_2bits = [[4, 2*(2**i), 2*(2**j)] for j in range(10) for i in range(10)]
layer_sizes_list_4bits = [[8, 2*(2**i), 2*(2**j)] for j in range(10) for i in range(10)]
layer_sizes_list_8bits = [[16, 2*(2**i), 2*(2**j)] for j in range(10) for i in range(10)]

In [23]:
# Calculate the performance for 2 hidden layers
# 2 bits
update_performance_dict(layer_sizes_list_2bits, performance_dict_2bits)
# 4 bits
update_performance_dict(layer_sizes_list_4bits, performance_dict_4bits)
# 8 bits
update_performance_dict(layer_sizes_list_8bits, performance_dict_8bits)

In [24]:
print('-'*7 + '2bits' + '-'*7)
print_performance_dict(performance_dict_2bits)
print('-'*7 + '4bits' + '-'*7)
print_performance_dict(performance_dict_4bits)
print('-'*7 + '8bits' + '-'*7)
print_performance_dict(performance_dict_8bits)

IOPub data rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_data_rate_limit`.

Current values:
ServerApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [25]:
# Save the data of the performance dicts
performance_dicts = [performance_dict_2bits, performance_dict_4bits, performance_dict_8bits]
for i in range(len(performance_dicts)):
    with open(f'./data/{2**(i+1)}bit_performance.txt', 'w') as f:
        f.write('parameters,performance\n')
        f.writelines([f'{x}\n' for x in sorted(list(performance_dicts[i].items()), key=lambda x: x[1], reverse=True)])
### You can run code below this

In [26]:
# Chose a few networks from the data file to run tests on
# General parameters
learning_rates = [0.001953125, 0.00390625, 0.0078125]
epochs_list = [5, 14, 19]

In [27]:
# Start with one hidden layer
transfer_functions = [logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_lin]

layer_sizes_list_2bits = [
    [4, 2],
    [4, 32],
    [4, 64],
]
layer_sizes_list_4bits = [
    [8, 64],
    [8, 2],
    [8, 1024],
]
layer_sizes_list_8bits = [
    [16, 16],
    [16, 64],
    [16, 4],
]

In [28]:
# Write a function to calculate the average performance
def update_average_performance_dict(layer_sizes_list, average_performance_dict, n_samples):
    cumulated_performance = 0
    for layer_sizes in layer_sizes_list:
        inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])
        layer_sizes.append(S)
        
        for rate in learning_rates:
                for epochs in epochs_list:
                    for i in range(n_samples):
                        reset_params()
                        train_network(learning_rate=rate, epochs=epochs)
                        cumulated_performance += performance()
                        
                    average_performance_dict.update({f'Layers: {layer_sizes}, Learning rate: {rate}, Epochs: {epochs}': cumulated_performance/n_samples})
                    cumulated_performance = 0

In [29]:
average_performance_dict_2bits = {}
average_performance_dict_4bits = {}
average_performance_dict_8bits = {}

In [30]:
update_average_performance_dict(layer_sizes_list_2bits, average_performance_dict_2bits, 500)
update_average_performance_dict(layer_sizes_list_4bits, average_performance_dict_4bits, 500)
update_average_performance_dict(layer_sizes_list_8bits, average_performance_dict_8bits, 500)

In [31]:
# Two hidden layers
transfer_functions = [logsig, logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_logsig, deriv_lin]

layer_sizes_list_2bits = [
    [4, 8, 2],
    [4, 2, 128],
    [4, 256, 32],
]
layer_sizes_list_4bits = [
    [8, 32, 64],
    [8, 1024, 64],
    [8, 2, 256],
]
layer_sizes_list_8bits = [
    [16, 8, 8],
    [16, 2, 1024],
    [16, 16, 4],
]

In [32]:
update_average_performance_dict(layer_sizes_list_2bits, average_performance_dict_2bits, 500)
update_average_performance_dict(layer_sizes_list_4bits, average_performance_dict_4bits, 500)
update_average_performance_dict(layer_sizes_list_8bits, average_performance_dict_8bits, 500)

In [33]:
print('----------2 bit average----------')
print_performance_dict(average_performance_dict_2bits)
print('----------4 bit average----------')
print_performance_dict(average_performance_dict_4bits)
print('----------8 bit average----------')
print_performance_dict(average_performance_dict_8bits)

----------2 bit average----------
Error: 11.343316577748705 | Layers: [4, 64, 3], Learning rate: 0.00390625, Epochs: 19
Error: 11.363598286883281 | Layers: [4, 64, 3], Learning rate: 0.001953125, Epochs: 19
Error: 11.371616150486174 | Layers: [4, 2, 3], Learning rate: 0.001953125, Epochs: 19
Error: 11.3742957382857 | Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 19
Error: 11.376807655677714 | Layers: [4, 32, 3], Learning rate: 0.00390625, Epochs: 19
Error: 11.396633630611525 | Layers: [4, 32, 3], Learning rate: 0.00390625, Epochs: 14
Error: 11.397898472068375 | Layers: [4, 64, 3], Learning rate: 0.00390625, Epochs: 14
Error: 11.400755617091951 | Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 14
Error: 11.40965966238322 | Layers: [4, 32, 3], Learning rate: 0.001953125, Epochs: 19
Error: 11.420508730546391 | Layers: [4, 32, 3], Learning rate: 0.0078125, Epochs: 19
Error: 11.424403951874211 | Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 19
Error: 11.43223332922953 |

In [34]:
# Save the average performance data
performance_dicts = [average_performance_dict_2bits, average_performance_dict_4bits, average_performance_dict_8bits]
for i in range(len(performance_dicts)):
    with open(f'./data/{2**(i+1)}bit_average_performance.txt', 'w') as f:
        f.write('parameters,performance\n')
        f.writelines([f'{x}\n' for x in sorted(list(performance_dicts[i].items()), key=lambda x: x[1], reverse=True)])

In [35]:
# Perform ceteris paribus test for 2 bist
# Read the average performance:
with open(f'./data/2bit_average_performance.txt', 'r') as f:
    average_performance = list(f)
    average_performance = [line.rstrip() for line in average_performance][1:]

In [36]:
# Chose for which networks to test
test_layers = [
    [4, 32, 3],
    [4, 256, 32, 3],
    [4, 2, 3],
]

test_learning_rates = [
    0.00390625,
    0.001953125,
    0.0078125,
]

test_epochs = [
    19,
    14,
    5,
]

def check_error_variation_over_paramater(const1, const2, name1, name2, name_test):
    print(f'----------Error variation across {name_test}----------')
    for i in range(len(const1)):
        print(f'Network {i+1}')
        for performance in average_performance:
            if f'{name1}: {const1[i]}' in performance and f'{name2}: {const2[i]}' in performance:
                print(performance)

check_error_variation_over_paramater(test_layers, test_learning_rates, 'Layers', 'Learning rate', 'epochs')
check_error_variation_over_paramater(test_layers, test_epochs, 'Layers', 'Epochs', 'learning rates')
check_error_variation_over_paramater(test_epochs, test_learning_rates, 'Epochs', 'Learning rate', 'epochs')

----------Error variation across epochs----------
Network 1
('Layers: [4, 32, 3], Learning rate: 0.00390625, Epochs: 5', 11.997607801682806)
('Layers: [4, 32, 3], Learning rate: 0.00390625, Epochs: 14', 11.396633630611525)
('Layers: [4, 32, 3], Learning rate: 0.00390625, Epochs: 19', 11.376807655677714)
Network 2
('Layers: [4, 256, 32, 3], Learning rate: 0.001953125, Epochs: 5', 21.527628325962123)
('Layers: [4, 256, 32, 3], Learning rate: 0.001953125, Epochs: 14', 16.696397469713183)
('Layers: [4, 256, 32, 3], Learning rate: 0.001953125, Epochs: 19', 14.458345798061867)
Network 3
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 5', 11.63426648485297)
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 14', 11.487549546490191)
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 19', 11.424403951874211)
----------Error variation across learning rates----------
Network 1
('Layers: [4, 32, 3], Learning rate: 0.0078125, Epochs: 19', 11.420508730546391)
('Layers: [4, 32, 3], Lea

In [37]:
# Perform the same test for 4 bits
# Read the average performance:
with open(f'./data/4bit_average_performance.txt', 'r') as f:
    average_performance = list(f)
    average_performance = [line.rstrip() for line in average_performance][1:]

In [38]:
# Define the parameters
test_layers = [
    [8, 2, 5],
    [8, 64, 5],
    [8, 32, 64, 5],
]

test_learning_rates = [
    0.0078125,
    0.001953125,
    0.00390625,
]

test_epochs = [
    19,
    14,
    5,
]

# Check error variation over epochs, learning rate and layers
check_error_variation_over_paramater(test_layers, test_learning_rates, 'Layers', 'Learning rate', 'epochs')
check_error_variation_over_paramater(test_layers, test_epochs, 'Layers', 'Epochs', 'learning rates')
check_error_variation_over_paramater(test_epochs, test_learning_rates, 'Epochs', 'Learning rate', 'epochs')

----------Error variation across epochs----------
Network 1
('Layers: [8, 2, 5], Learning rate: 0.0078125, Epochs: 5', 11.61339954796706)
('Layers: [8, 2, 5], Learning rate: 0.0078125, Epochs: 14', 11.51920929809937)
('Layers: [8, 2, 5], Learning rate: 0.0078125, Epochs: 19', 11.396692994716144)
Network 2
('Layers: [8, 64, 5], Learning rate: 0.001953125, Epochs: 5', 24.528437264426405)
('Layers: [8, 64, 5], Learning rate: 0.001953125, Epochs: 14', 11.446804599845231)
('Layers: [8, 64, 5], Learning rate: 0.001953125, Epochs: 19', 11.356593633034988)
Network 3
('Layers: [8, 32, 64, 5], Learning rate: 0.00390625, Epochs: 5', 18.81755778940776)
('Layers: [8, 32, 64, 5], Learning rate: 0.00390625, Epochs: 14', 12.568354135528049)
('Layers: [8, 32, 64, 5], Learning rate: 0.00390625, Epochs: 19', 11.964359157241388)
----------Error variation across learning rates----------
Network 1
('Layers: [8, 2, 5], Learning rate: 0.0078125, Epochs: 19', 11.396692994716144)
('Layers: [8, 2, 5], Learning r

In [39]:
# Perform the same test for 8 bits
# Read the average performance:
with open(f'./data/8bit_average_performance.txt', 'r') as f:
    average_performance = list(f)
    average_performance = [line.rstrip() for line in average_performance][1:]

In [40]:
# Define the parameters
test_layers = [
    [16, 4, 9],
    [16, 64, 9],
    [16, 2, 1024, 9],
]

test_learning_rates = [
    0.0078125,
    0.001953125,
    0.00390625,
]

test_epochs = [
    19,
    14,
    5,
]

# Check error variation over epochs, learning rate and layers
check_error_variation_over_paramater(test_layers, test_learning_rates, 'Layers', 'Learning rate', 'epochs')
check_error_variation_over_paramater(test_layers, test_epochs, 'Layers', 'Epochs', 'learning rates')
check_error_variation_over_paramater(test_epochs, test_learning_rates, 'Epochs', 'Learning rate', 'epochs')

----------Error variation across epochs----------
Network 1
('Layers: [16, 4, 9], Learning rate: 0.0078125, Epochs: 5', 11.61402713385054)
('Layers: [16, 4, 9], Learning rate: 0.0078125, Epochs: 14', 11.473610884310123)
('Layers: [16, 4, 9], Learning rate: 0.0078125, Epochs: 19', 11.447346629486992)
Network 2
('Layers: [16, 64, 9], Learning rate: 0.001953125, Epochs: 5', 24.81542572790746)
('Layers: [16, 64, 9], Learning rate: 0.001953125, Epochs: 14', 11.464069798002816)
('Layers: [16, 64, 9], Learning rate: 0.001953125, Epochs: 19', 11.35731566114027)
Network 3
('Layers: [16, 2, 1024, 9], Learning rate: 0.00390625, Epochs: 5', 18.84386124786892)
('Layers: [16, 2, 1024, 9], Learning rate: 0.00390625, Epochs: 14', 12.587965883428224)
('Layers: [16, 2, 1024, 9], Learning rate: 0.00390625, Epochs: 19', 11.969446662783584)
----------Error variation across learning rates----------
Network 1
('Layers: [16, 4, 9], Learning rate: 0.0078125, Epochs: 19', 11.447346629486992)
('Layers: [16, 4, 9

In [41]:
# Come back and test all the 2 bit network networks
# over the learning rates
# Read the data
with open(f'./data/2bit_average_performance.txt', 'r') as f:
    average_performance = list(f)
    average_performance = [line.rstrip() for line in average_performance][1:]
    
# Define parameters
test_layers = [
    [4, 2, 3],
    [4, 32, 3],
    [4, 64, 3],
    [4, 8, 2, 3],
    [4, 2, 128, 3],
    [4, 256, 32, 3],
]

test_epochs = [
    5, 
    14, 
    19
]

In [42]:
# Order network performance for all the corresponding networks
for i, layers in enumerate(test_layers):
    for j, epochs in enumerate(test_epochs):
        print(f'Network {i*len(test_epochs) + j + 1} - Layers: {layers} | Epochs: {epochs}')
        for performance in average_performance:
                    if f'Layers: {layers}' in performance and f'Epochs: {epochs}' in performance:
                        print(performance)

Network 1 - Layers: [4, 2, 3] | Epochs: 5
('Layers: [4, 2, 3], Learning rate: 0.001953125, Epochs: 5', 24.775144624157907)
('Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 5', 12.01129447764442)
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 5', 11.63426648485297)
Network 2 - Layers: [4, 2, 3] | Epochs: 14
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 14', 11.487549546490191)
('Layers: [4, 2, 3], Learning rate: 0.001953125, Epochs: 14', 11.446780129613042)
('Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 14', 11.400755617091951)
Network 3 - Layers: [4, 2, 3] | Epochs: 19
('Layers: [4, 2, 3], Learning rate: 0.0078125, Epochs: 19', 11.424403951874211)
('Layers: [4, 2, 3], Learning rate: 0.00390625, Epochs: 19', 11.3742957382857)
('Layers: [4, 2, 3], Learning rate: 0.001953125, Epochs: 19', 11.371616150486174)
Network 4 - Layers: [4, 32, 3] | Epochs: 5
('Layers: [4, 32, 3], Learning rate: 0.001953125, Epochs: 5', 24.730839115956222)
('Layers: [4, 32, 3], Lea

In [43]:
# TODO: test stable network for whole range of epochs and record results
# TODO: test extremely simplified network structures see beneath as well
# TODO: test an extremely complexe network structure (just a lot of layers)
# TODO: test stable network performance on hard lim transfer functions
# TODO: test stable network performance but training it with a hardlim transfer function last layer
# TODO: redo all this with symetrical network

In [44]:
# Testing network structures trained with huge numbers of epochs
# Define layers
layer_sizes = [4, 32]

# Get inputs and outputs
inputs, outputs, S = gen_inputs_outputs(layer_sizes[0])

# Append last layer size
layer_sizes = np.array(layer_sizes + [S], dtype=int)

# Define transfer functions
transfer_functions = [logsig, lin]
deriv_transfer_functions = [deriv_logsig, deriv_lin]

In [47]:
# Test for epochs going from 100 to 5000
for i in range(1, 51):
    # Reset all network parameters
    reset_params()
    
    print(f'-----Epochs: {i*100}-----')
    
    # Train the network
    train_network(learning_rate=0.00390625, epochs=i*100)

    # Get performance
    print(performance())

-----Epochs: 100-----
10.658642018811928
-----Epochs: 200-----
9.788723652175616
-----Epochs: 300-----
9.225707011910234
-----Epochs: 400-----
9.24702045816121
-----Epochs: 500-----
9.109805483277741
-----Epochs: 600-----
8.702733332929716
-----Epochs: 700-----
8.69729758027637
-----Epochs: 800-----
9.012818495164924
-----Epochs: 900-----
8.224303545355763
-----Epochs: 1000-----
8.46033528047256
-----Epochs: 1100-----
8.001657573123683
-----Epochs: 1200-----
8.08523168345353
-----Epochs: 1300-----
7.297266812210767
-----Epochs: 1400-----
6.090375118190548
-----Epochs: 1500-----
6.3760222126849735
-----Epochs: 1600-----
7.454710762160716
-----Epochs: 1700-----
6.016528339431915
-----Epochs: 1800-----
5.3690559379327425
-----Epochs: 1900-----
5.454565248538879
-----Epochs: 2000-----
4.331421359732537
-----Epochs: 2100-----
4.376222579665006
-----Epochs: 2200-----
3.949197975006062
-----Epochs: 2300-----
4.4042379946949435
-----Epochs: 2400-----
3.9493321479473438
-----Epochs: 2500-----
3